In [1]:
# Importação de bibliotecas
import pandas as pd

import googledriver as gdriver

In [2]:
dataset_name = 'iLPF_SPD'
raiz = '/home/eduardo/repositorios/IS_Agro/databases'

### Bronze

In [3]:
id_arquivo = '1EA8tpKJvgyyMYGnK8DtsBDnE1DRtcWBP0ab6dYGTZwM' # ID adquirida na URL
url_gdrive = f'https://docs.google.com/spreadsheets/d/{id_arquivo}/export?format=xlsx&t?gid=0#gid=0' # Endereço específico para exportação de dados planilhares

saida = f'{raiz}/bronze_{dataset_name}.xlsx'

gdriver.download(url_gdrive, saida, None)

'/home/eduardo/repositorios/IS_Agro/databases/bronze_iLPF_SPD.xlsx'

### Prata

In [4]:
# Abas das tabelas na planilha bronze
tabelas = pd.ExcelFile(f'{raiz}/bronze_{dataset_name}.xlsx').sheet_names

In [6]:
for area in tabelas:
        df = (
                pd.read_excel(f'{raiz}/bronze_{dataset_name}.xlsx',
                            sheet_name=area,
                            header=2,
                            nrows=27,
                            na_values=pd.NA
                        )
                .drop(columns=['regiao', 'uf'])
                .dropna(axis='columns', thresh=10)
                .melt(id_vars='geocodigo',
                      var_name='data',
                      value_name='valor'
                      )
                .dropna(subset='valor')
                .query('valor != 0') # Exclusão de registros que possuam valor zerado
                .assign(data = lambda df: pd.to_datetime(df['data'], format='%Y')) # Conversão de número inteiro para formato de data
                .sort_values(by=['geocodigo', 'data']) # Organização dos dados
                .set_index(['geocodigo', 'data'])
        )
        df.to_parquet(f'{raiz}/prata_{area}.parquet')